In [9]:
import numpy as np
import pandas as pd

def Load_PAF(filepath, filter_thresh = 90.0):
    lines = open(filepath).readlines()
    header = ['Query','Qlen','QStart','QEnd','Orientation','Subject','SLen',
              'SStart','SEnd','Matches','AlignLength','MAPQ']
    op = []
    for l in lines:
        l = l.split('\t')[:12]
        op.append(dict(zip(header, l)))
    df = pd.DataFrame(op)
    df[['Qlen','QStart','QEnd','SLen','SStart',
        'SEnd','Matches','AlignLength','MAPQ']] = df[['Qlen','QStart','QEnd','SLen','SStart',
                                                      'SEnd','Matches','AlignLength','MAPQ']].astype('int')
    df['PIdent'] = df['Matches']/df[['Qlen','SLen']].min(axis = 1)*100
    df = df[df['PIdent'] >= filter_thresh]
    df = df.loc[df.groupby(['Query'])['PIdent'].idxmax()]
    return df

In [2]:
blast_path = '/Users/harihara/Mount-2/Hot_Springs_Analysis/Scripts/Strain-Analysis/Single_Cell_Gabe/YSG_1331.txt'

In [27]:
YSG_O_allele = ['Ga0374759_019_7127_10582','Ga0393479_012_24154_27609','Ga0374745_146_276_3749']

In [3]:
df_blast = pd.read_csv(blast_path, names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 
                                            'gapopen', 'qstart', 'qend', 'sstart', 'send', 
                                            'evalue', 'bitscore', 'qlen', 'slen'], sep = "\t")

In [51]:
df_blast['Matches'] = df_blast['length'] - df_blast['mismatch'] - df_blast['gapopen']
df_blast['PIdent'] = df_blast['Matches']/df_blast['qlen']*100.0
df_blast_filter = df_blast[df_blast['PIdent'] >= 90]
df_blast_filter = df_blast_filter.loc[df_blast_filter.groupby(['qseqid'])['PIdent'].idxmax()]

In [10]:
paf_path = '/Users/harihara/Mount/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/test_runs_recruitment_v2/O_cluster_test/MS65.paf'

In [37]:
df_paf = Load_PAF(paf_path, 90.0)

In [52]:
reads_blast = set(df_blast_filter['qseqid'].tolist())
reads_minimap2 = set(df_paf['Query'].tolist())

print(len(reads_blast))
print(len(reads_minimap2))

56555
44912


In [53]:
common = len(reads_blast&reads_minimap2)
only_blast = len(reads_blast - reads_minimap2)
only_minimap2 = len(reads_minimap2 - reads_blast)

In [54]:
common

44912

In [55]:
only_blast

11643

In [56]:
only_minimap2

0

In [46]:
df_blast_O_Allele = df_blast_filter[df_blast_filter['sseqid'].isin(YSG_O_allele)]
df_paf_O_Allele = df_paf[df_paf['Subject'].isin(YSG_O_allele)]

,Query,Qlen,QStart,QEnd,Orientation,Subject,SLen,SStart,SEnd,Matches,AlignLength,MAPQ,PIdent
